In [1]:
import h2o

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 30 mins
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.5
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_renat_m0uj1h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.725 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
data = h2o.import_file("http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
train, valid, test = data.split_frame([0.8,0.1], seed = 69)

In [6]:
print ("%d/%d/%d" % (train.nrows,valid.nrows,test.nrows))

35255/4272/4451


In [7]:
y = "IsArrDelayed"
ignoreFields = [
    "ArrDelay", "DepDelay", "carrierDelay","WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "IsDepDelayed",
    "IsArrDelayed", "ActualElapsedTime", "ArrTime", "TailNum"
]
x= [i for i in train.names if i not in ignoreFields]

In [8]:
nfolds=5
train2 = train.rbind (valid)

In [9]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [11]:
m_GLM = H2OGeneralizedLinearEstimator(
    family = "binomial",
    model_id = "glm_def",
    nfolds = nfolds,
    fold_assignment = "Modulo",
    keep_cross_validation_predictions = True
)
m_GLM.train (x,y,train2)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
m_GBM = H2OGradientBoostingEstimator(
    model_id = "gbm_def",
    nfolds = nfolds,
    fold_assignment = "Modulo",
    keep_cross_validation_predictions = True
)
m_GBM.train (x,y,train2)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [13]:
m_RF = H2ORandomForestEstimator (
    model_id = "rf_def",
    nfolds = nfolds,
    fold_assignment = "Modulo",
    keep_cross_validation_predictions = True
)
m_RF.train (x,y,train2)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [14]:
models = [m_GLM.model_id, m_GBM.model_id, m_RF.model_id]

In [15]:
m_SE = H2OStackedEnsembleEstimator (model_id = "SE_glm_gbm_rf",
                                    base_models = models)
m_SE.train(x,y,train2)

stackedensemble Model Build progress: |███████████████████████████████████| 100%


In [17]:
import pandas as pd

In [18]:
all_models = [m_GLM, m_GBM, m_RF, m_SE]

In [19]:
names = ["GLM","GBM","RF","SE"]

In [20]:
pd.Series(map(lambda x: x.logloss(), all_models), names)

GLM    0.562567
GBM    0.502611
RF     0.490367
SE     0.237360
dtype: float64

In [21]:
pd.Series(map(lambda x: x.auc(),all_models), names)

GLM    0.775300
GBM    0.851012
RF     0.845417
SE     0.989179
dtype: float64

In [22]:
pd.Series (map(lambda x: x.auc (xval=True),all_models), names)

GLM    0.768367
GBM    0.810133
RF     0.848386
SE          NaN
dtype: float64

In [25]:
test_perf = list (map(lambda x: x.model_performance(test), all_models))

In [26]:
pd.Series(map(lambda p: p.logloss(), test_perf), names)

GLM    0.572595
GBM    0.536000
RF     0.475277
SE     0.470408
dtype: float64

In [27]:
pd.Series(map(lambda p: p.auc(), test_perf), names)

GLM    0.761364
GBM    0.808359
RF     0.854386
SE     0.853944
dtype: float64

In [16]:
mA= H2OAutoML (max_runtime_secs =30)
mA.train (["ArrTime", "ActualElapsedTime", "ArrDelay", "carrierDelay", "WeatherDelay", 
           "NASDelay", "SecurityDelay", "LateAircraftDelay", "IsDepDelayed"], "class", train)

H2OValueError: Column class does not exist in the training frame

In [9]:
from h2o.automl import H2OAutoML

In [18]:
p = ma.leader.predict(test)

NameError: name 'ma' is not defined

In [17]:
p

NameError: name 'ma' is not defined

In [19]:
ma.leader.model_performance (test)

NameError: name 'ma' is not defined